# Cálculo de intervalos de confianza (IC95%) de la edad por país

En esta sección se calculan los intervalos de confianza del 95% (IC95%) para la edad media de los pacientes en cada país (Brasil, México y España).  
Este análisis se añade como corrección solicitada por la tutora, para reforzar la representatividad de la muestra y aportar mayor solidez estadística en la Tabla 1 de Resultados.


In [1]:
import pandas as pd
import numpy as np
import os

# --- 1) Cargar dataset ---
df = pd.read_csv("outputs/dataset_ready.csv")

# --- 2) Seleccionar columnas relevantes ---
# Ajusta los nombres de columnas según tu dataset
df = df.rename(columns={"Age (years)": "Edad", "Country": "Pais"})

# --- 3) Limpiar valores nulos o no numéricos ---
df = df.dropna(subset=["Edad"])
df["Edad"] = pd.to_numeric(df["Edad"], errors="coerce")
df = df.dropna(subset=["Edad"])

# --- 4) Calcular estadísticos e IC95% por país ---
res = df.groupby("Pais")["Edad"].agg(
    n="count",
    Media="mean",
    DE=lambda x: x.std(ddof=1)
).reset_index()

res["SEM"] = res["DE"] / np.sqrt(res["n"])
res["IC95_inf"] = res["Media"] - 1.96*res["SEM"]
res["IC95_sup"] = res["Media"] + 1.96*res["SEM"]

# --- 5) Redondear ---
res_round = res.copy()
cols = ["Media","DE","IC95_inf","IC95_sup"]
res_round[cols] = res_round[cols].round(2)

# --- 6) Mostrar resultado ---
print(res_round[["Pais","n","Media","DE","IC95_inf","IC95_sup"]])

# --- 7) Exportar ---
os.makedirs("outputs/tablas", exist_ok=True)
res_round.to_csv("outputs/tablas/tabla1_ic95_edad.csv", index=False)
res_round.to_excel("outputs/tablas/tabla1_ic95_edad.xlsx", index=False)

# --- 8) Generar nota para pegar bajo Tabla 1 ---
def fmt_row(r):
    return f"{r['Pais']} {r['Media']:.1f} ({r['IC95_inf']:.1f}–{r['IC95_sup']:.1f})"

nota = "; ".join([fmt_row(r) for _, r in res_round.iterrows()])
print("\nNota para la tabla:")
print(f"Edad media (IC95%): {nota}. Muestra clínica; no pretende ser representativa de la población general con LES.")


     Pais    n  Media     DE  IC95_inf  IC95_sup
0  Brazil  143  39.16   9.40     37.62     40.70
1  Mexico   70  35.91  12.87     32.90     38.93
2   Spain   77  52.17  12.68     49.34     55.00

Nota para la tabla:
Edad media (IC95%): Brazil 39.2 (37.6–40.7); Mexico 35.9 (32.9–38.9); Spain 52.2 (49.3–55.0). Muestra clínica; no pretende ser representativa de la población general con LES.


### Nota al pie para pegar debajo de la tabla

Edad media (IC95%): Brasil 39.2 (37.6–40.7); México 35.9 (32.9–38.9); España 52.2 (49.3–55.0). Muestra clínica; no pretende ser representativa de la población general con LES.

### Explicación del procedimiento
1. Se seleccionaron las columnas `Edad` y `Pais` del dataset final (`dataset_ready.csv`).  
2. Se eliminaron registros con valores nulos o no numéricos en la edad.  
3. Se calcularon los estadísticos básicos por país:
   - **n**: número de pacientes.  
   - **Media**: edad media.  
   - **DE**: desviación estándar.  
4. A partir del error estándar de la media (SEM = DE/√n), se calcularon los intervalos de confianza del 95%:
   - Límite inferior = Media – 1.96 × SEM  
   - Límite superior = Media + 1.96 × SEM  
5. Los resultados se redondearon a 2 decimales y se exportaron en formatos `.csv` y `.xlsx` para su documentación.
